In [56]:
import pickle
import pandas as pd
from datetime import datetime
import logging
import csv

In [ ]:
with open('customer_orders.pkl', 'rb') as f:
    customer_orders = pickle.load(f)

if customer_orders:
    customer = customer_orders[0]
    print("Customer keys and types:")
    for k, v in customer.items():
        print(f"  {k}: {type(v)}")
    if 'orders' in customer and customer['orders']:
        order = customer['orders'][0]
        print("\nOrder keys and types:")
        for k, v in order.items():
            print(f"  {k}: {type(v)}")
        if 'items' in order and order['items']:
            item = order['items'][0]
            print("\nItem keys and types:")
            for k, v in item.items():
                print(f"  {k}: {type(v)}")

Customer keys and types:
  id: <class 'int'>
  name: <class 'str'>
  registration_date: <class 'str'>
  orders: <class 'list'>

Order keys and types:
  order_id: <class 'int'>
  order_date: <class 'str'>
  shipping_address: <class 'str'>
  order_total_value: <class 'int'>
  items: <class 'list'>


In [51]:
for i, cust in enumerate(customer_orders[:2]):
    print(f"Customer {i+1}:")
    print(cust)
    print('-' * 40)

Customer 1:
{'id': 1, 'name': 'Customer 1', 'registration_date': '2022-12-31 04:19:19', 'orders': [{'order_id': 84, 'order_date': '2024-02-27 21:24:16', 'shipping_address': '840 Main St, City 8', 'order_total_value': 0, 'items': []}, {'order_id': None, 'order_date': '2025-05-03 08:09:20', 'shipping_address': '533 Main St, City 9', 'order_total_value': 2765.8, 'items': [{'item_id': 1, 'product_name': 'Item 1 for Order 2', 'category': 4, 'price': 52.7, 'quantity': 2}, {'item_id': 2, 'product_name': 'Item 2 for Order 2', 'category': 5, 'price': 498.45, 'quantity': 5}, {'item_id': 3, 'product_name': 'Item 3 for Order 2', 'category': 5, 'price': 168.15, 'quantity': 1}]}, {'order_id': 3, 'order_date': '2023-09-06 00:42:50', 'shipping_address': '588 Main St, City 3', 'order_total_value': 0, 'items': [{'item_id': 1, 'product_name': 'Item 1 for Order 3', 'category': 1, 'price': '$377.96', 'quantity': 1}]}]}
----------------------------------------
Customer 2:
{'id': 2, 'name': 'Customer 2', 're

In [58]:
class CustomerDataExtractor:
    def __init__(self, pickle_path='customer_orders.pkl', vip_path='vip_customers.txt', log_path='skipped_entries.log'):
        self.pickle_path = pickle_path
        self.vip_path = vip_path
        self.log_path = log_path
        self._setup_logger()
        self.vip_ids = self._load_vip_ids()
        self.category_map = {
            1: 'Electronics',
            2: 'Apparel',
            3: 'Books',
            4: 'Home Goods'
        }

    def _setup_logger(self):
        logging.basicConfig(
            filename=self.log_path,
            level=logging.INFO,
            filemode='w',
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def _load_vip_ids(self):
        try:
            with open(self.vip_path, 'r') as f:
                return {int(line.strip()) for line in f if line.strip()}
        except FileNotFoundError:
            logging.info(f"VIP file not found at {self.vip_path}. Continuing without VIPs.")
            return set()

    def _normalize_category(self, cat_val):
        try:
            if isinstance(cat_val, str):
                return {
                    'electronics': 'Electronics',
                    'apparel': 'Apparel',
                    'books': 'Books',
                    'home goods': 'Home Goods'
                }.get(cat_val.strip().lower(), 'Misc')
            elif int(cat_val) in self.category_map:
                return self.category_map[int(cat_val)]
        except Exception:
            pass
        return 'Misc'

    def transform_to_dataframe(self):
        try:
            with open(self.pickle_path, 'rb') as f:
                customer_orders = pickle.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"Pickle file not found: {self.pickle_path}")

        flat_data = []

        for customer in customer_orders:
            cust_id = customer.get('id', 0)
            customer_name = customer.get('name', "N/A")
            registration_date = customer.get('registration_date', "1970-01-01")
            is_vip = bool(cust_id in self.vip_ids)

            # If 'orders' missing, add a row with N/A/defaults
            if 'orders' not in customer or customer['orders'] is None:
                flat_data.append({
                    'customer_id': int(cust_id) if isinstance(cust_id, int) else 0,
                    'customer_name': str(customer_name) if isinstance(customer_name, str) else "N/A",
                    'registration_date': registration_date,
                    'is_vip': is_vip,
                    'order_id': 0,
                    'order_date': "1970-01-01",
                    'product_id': 0,
                    'product_name': "N/A",
                    'category': "N/A",
                    'unit_price': 0.0,
                    'item_quantity': 0,
                    'total_item_price': 0.0,
                    'total_order_value_percentage': 0.0
                })
                continue

            for order in customer['orders']:
                if not isinstance(order, dict):
                    flat_data.append({
                        'customer_id': int(cust_id) if isinstance(cust_id, int) else 0,
                        'customer_name': str(customer_name) if isinstance(customer_name, str) else "N/A",
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': 0,
                        'order_date': "1970-01-01",
                        'product_id': 0,
                        'product_name': "N/A",
                        'category': "Misc",
                        'unit_price': 0.0,
                        'item_quantity': 0,
                        'total_item_price': 0.0,
                        'total_order_value_percentage': 0.0
                    })
                    continue

                order_id = order.get('order_id', 0)
                order_date = order.get('order_date', "1970-01-01")
                items = order.get('items', [])

                # If order_id is missing or items is empty, add a row with N/A/defaults
                if not order_id or not items:
                    flat_data.append({
                        'customer_id': int(cust_id) if isinstance(cust_id, int) else 0,
                        'customer_name': str(customer_name) if isinstance(customer_name, str) else "N/A",
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': int(order_id) if str(order_id).isdigit() else 0,
                        'order_date': order_date,
                        'product_id': 0,
                        'product_name': "N/A",
                        'category': "Misc",
                        'unit_price': 0.0,
                        'item_quantity': 0,
                        'total_item_price': 0.0,
                        'total_order_value_percentage': 0.0
                    })
                    continue

                order_items = []
                order_total = 0.0

                for item in items:
                    try:
                        # Handle FREE quantity
                        quantity_raw = item.get('quantity', 0)
                        if isinstance(quantity_raw, str) and quantity_raw.strip().upper() == 'FREE':
                            quantity = 0
                        else:
                            try:
                                quantity = int(quantity_raw)
                            except Exception:
                                quantity = 0

                        # Handle price
                        price_raw = item.get('price', 0.0)
                        if price_raw is None or (isinstance(price_raw, str) and price_raw.strip().upper() == 'NONE'):
                            price = 0.0
                        elif isinstance(price_raw, str):
                            try:
                                price = float(price_raw.replace('$', '').replace(',', '').strip())
                            except Exception:
                                price = 0.0
                        else:
                            try:
                                price = float(price_raw)
                            except Exception:
                                price = 0.0

                        item_id = item.get('item_id', 0)
                        if not str(order_id).isdigit() or not str(item_id).isdigit():
                            continue

                        item_total = price * quantity
                        order_items.append((item, item_total, price, quantity))
                        order_total += item_total
                    except Exception:
                        continue

                for item, item_total, price, quantity in order_items:
                    category_val = item.get('category', None)
                    if isinstance(category_val, int) and category_val in self.category_map:
                        category_str = self.category_map[category_val]
                    else:
                        category_str = 'Misc'
                    flat_data.append({
                        'customer_id': int(cust_id) if isinstance(cust_id, int) else 0,
                        'customer_name': str(customer_name) if isinstance(customer_name, str) else "N/A",
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': int(order_id) if str(order_id).isdigit() else 0,
                        'order_date': order_date,
                        'product_id': int(item.get('item_id', 0)) if str(item.get('item_id', 0)).isdigit() else 0,
                        'product_name': item.get('product_name', "N/A") if isinstance(item.get('product_name', "N/A"), str) else "N/A",
                        'category': category_str if isinstance(category_str, str) else "N/A",
                        'unit_price': float(price),
                        'item_quantity': int(quantity),
                        'total_item_price': float(item_total),
                        'total_order_value_percentage': float((item_total / order_total * 100) if order_total else 0.0)
                    })

        df = pd.DataFrame(flat_data)

        # Set types as requested, fill missing/invalid with N/A or default
        df['customer_id'] = df['customer_id'].fillna(0).astype(int)
        df['customer_name'] = df['customer_name'].fillna("N/A").astype(str)
        df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce').fillna(pd.Timestamp("1970-01-01"))
        df['is_vip'] = df['is_vip'].fillna(False).astype(bool)
        df['order_id'] = df['order_id'].fillna(0).astype(int)
        df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce').fillna(pd.Timestamp("1970-01-01"))
        df['product_id'] = df['product_id'].fillna(0).astype(int)
        df['product_name'] = df['product_name'].fillna("N/A").astype(str)
        df['category'] = df['category'].fillna("N/A").astype(str)
        df['unit_price'] = df['unit_price'].fillna(0.0).astype(float)
        df['item_quantity'] = df['item_quantity'].fillna(0).astype(int)
        df['total_item_price'] = df['total_item_price'].fillna(0.0).astype(float)
        df['total_order_value_percentage'] = df['total_order_value_percentage'].fillna(0.0).astype(float)

        return df.sort_values(['customer_id', 'order_id', 'product_id']).reset_index(drop=True)


In [59]:
extractor = CustomerDataExtractor()
df = extractor.transform_to_dataframe()
print(df.head())
df.to_csv('customer_orders_transformed.csv', index=False)
print("DataFrame saved to 'customer_orders_transformed.csv'")

   customer_id customer_name   registration_date  is_vip  order_id  \
0            1    Customer 1 2022-12-31 04:19:19    True         0   
1            1    Customer 1 2022-12-31 04:19:19    True         3   
2            1    Customer 1 2022-12-31 04:19:19    True        84   
3            2    Customer 2 2022-05-27 00:23:28   False         3   
4            2    Customer 2 2022-05-27 00:23:28   False         5   

           order_date  product_id        product_name     category  \
0 2025-05-03 08:09:20           0                 N/A         Misc   
1 2023-09-06 00:42:50           1  Item 1 for Order 3  Electronics   
2 2024-02-27 21:24:16           0                 N/A         Misc   
3 2023-01-01 07:32:50           0                 N/A         Misc   
4 2024-08-28 19:37:56           1  Item 1 for Order 5         Misc   

   unit_price  item_quantity  total_item_price  total_order_value_percentage  
0        0.00              0              0.00                      0.000000  


In [ ]:
with open('customer_orders_transformed.csv', 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    csv_row_count = sum(1 for row in csv_reader) - 1

with open('customer_orders.pkl', 'rb') as f:
    customer_orders_pkl = pickle.load(f)

df_from_pkl = extractor.transform_to_dataframe()
pkl_row_count = len(df_from_pkl)

print(f"CSV row count: {csv_row_count}")
print(f"Pickle flattened row count: {pkl_row_count}")
print("Counts match!" if csv_row_count == pkl_row_count else "Counts do NOT match!")

CSV row count: 193
Pickle flattened row count: 193
Counts match!
